In [1]:
from uart import *
import time
import numpy as np
import glob
import platform

In [2]:
def port_list():
    os_name = platform.system()
    if "Windows" in os_name:
        print("Current OS: Windows")
        ports = ['COM%s' %(i+1) for i in range(256)]
    elif "Linux"in os_name:
        print("Current OS: Linux")
        ports = glob.glob('/dev/tty[A-Za-z]*')
    elif "Darwin" in os_name:
        print("Current OS: Mac")
        ports = glob.glob('/dev/tty.*')
    result = []
    for p in ports:
        try:
            s = serial.Serial(p)
            s.close()
            result.append(p)
        except (OSError, serial.SerialException):
            pass
    print(result)
    return result

In [3]:
plist = port_list()

Current OS: Windows
['COM1', 'COM5']


In [4]:
for pname in plist:
    try:
        SU = UART(str(pname))
        print("%s port connected!!" %(pname))
        break
    except serial.SerialException:
        print("%s port cannot be connected." %(pname))

COM1 port cannot be connected.
COM5 port connected!!


In [5]:
SU.su_flush_buffer()

In [6]:
MODE_TEST_HELLO = 0x01
MODE_TEST_ECHO = 0x02
MODE_STORE_RAM = 0x03
MODE_LOAD_RAM = 0x04
MODE_STORE_TEST = 0x05
MODE_LOAD_TEST = 0x06

In [7]:
result = SU.test_hello()
print(result)

[   OK   ] Mode
b'  Hello, World! '


In [8]:
result = SU.test_echo('this is echo')
print("result:", result)

[   OK   ] Mode
result: b'this is echo    '


In [9]:
result = SU.mem_test()
print(result)

[   OK   ] Mode
b' MEM DONE       '


In [10]:
file_name = 'testfile.txt'
base_addr= 0x00000000
words_to_send = 2304

In [11]:
SU.store_file(file_name, base_addr, words_to_send)
print("-------------------------------------")
data = SU.verify_file(base_addr, words_to_send)
print("-------------------------------------")
SU.check_result(file_name, data, words_to_send)

[   OK   ] Mode
[  OK  ] Wait_CMD
[  OK  ] Receive
b' Store complete '
-------------------------------------
[   OK   ] Mode
b'  Waiting_CMD   '
[  OK  ] Wait_CMD
[   OK   ] CMD_Receive
-------------------------------------
Correct!!


True

In [13]:
print(SU.ser.out_waiting)
print(SU.ser.in_waiting)

0
0
